This notebook collectes 1000 light curves of each variable type from Xiaodian's ZTF labeled dataset

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import sqlalchemy as db

Xiaodian's ZTF labeled dataset feature and label table

In [ ]:
widths = (8,7,4,13,43)
header_pd = pd.read_fwf('Labeled_data.txt', widths = widths,skiprows=7, nrows=27)
labeled_data = pd.read_csv('Labeled_data.txt', header=None, delim_whitespace=True, skiprows=36) # extract data
labeled_data.columns = header_pd.iloc[:,3]
labeled_data.head()

In [ ]:
label = labeled_data.Type.unique()
label = np.delete(label, np.where(label == 'CEPII')) # CEPII has very few example so we ignore it
print(label)

Download light curve csv file for fetaure and label row for 1000 of each label type. Store lightcurves in an SQLite database

In [ ]:
engine = db.create_engine('sqlite:///save_LC_1000each.db', echo=False)
sqlite_connection = engine.connect()
for variable_type in label:
    print(str(variable_type))
    df = labeled_data[labeled_data['Type']==variable_type]
    for i in range(0,1000):
        k = df.index[i]
        url = 'http://variables.cn:88/seldataz.php?SourceID=' + str(df.SourceID[k])   
        try:
            lc = pd.read_csv(url, header='infer')
            lc['Type'] = variable_type
            lc['ID'] = labeled_data.ID[k]
        except:
            continue
        sqlite_table = labeled_data.ID[k]
        lc.to_sql(sqlite_table, sqlite_connection, if_exists='replace')
        print(i)